In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

from torch.autograd import Variable

from torchvision import datasets, transforms

from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

In [2]:
batch_size = 64

train_dataset = datasets.MNIST(root='../data/', train=True, transform= transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='../data/', train=False, transform= transforms.ToTensor())

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.L1 = nn.Linear(784, 520)
        self.L2 = nn.Linear(520, 320)
        self.L3 = nn.Linear(320, 240)
        self.L4 = nn.Linear(240, 120)
        self.L5 = nn.Linear(120, 10)
        
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.L1(x))
        x = F.relu(self.L2(x))
        x = F.relu(self.L3(x))
        x = F.relu(self.L4(x))
        return self.L5(x)
    
model = Net()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

Let's use 2 GPUs!


In [4]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data),Variable(target)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t Loss:{:.6f}'.format(epoch, batch_idx*len(data)
                                                                           ,len(train_loader.dataset),
                                                                          100.*batch_idx /len(train_loader),
                                                                          loss.data[0]))

In [5]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True),Variable(target)
        data, target = data.to(device), target.to(device)
        
        output = model(data)
        
        test_loss += criterion(output, target).data[0]
        
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
for epoch in range(1, 10):
    train(epoch)
test()

/home/hsue/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1 [0/60000 (0%)]	 Loss:2.302313
Train Epoch: 1 [6400/60000 (11%)]	 Loss:2.299247
Train Epoch: 1 [12800/60000 (21%)]	 Loss:2.290041
Train Epoch: 1 [19200/60000 (32%)]	 Loss:2.300921
Train Epoch: 1 [25600/60000 (43%)]	 Loss:2.295409
Train Epoch: 1 [32000/60000 (53%)]	 Loss:2.280081
Train Epoch: 1 [38400/60000 (64%)]	 Loss:2.262786
Train Epoch: 1 [44800/60000 (75%)]	 Loss:2.206924
Train Epoch: 1 [51200/60000 (85%)]	 Loss:2.086842
Train Epoch: 1 [57600/60000 (96%)]	 Loss:1.657949
Train Epoch: 2 [0/60000 (0%)]	 Loss:1.547017
Train Epoch: 2 [6400/60000 (11%)]	 Loss:1.145486
Train Epoch: 2 [12800/60000 (21%)]	 Loss:0.998540
Train Epoch: 2 [19200/60000 (32%)]	 Loss:0.654078
Train Epoch: 2 [25600/60000 (43%)]	 Loss:0.585317
Train Epoch: 2 [32000/60000 (53%)]	 Loss:0.655989
Train Epoch: 2 [38400/60000 (64%)]	 Loss:0.429679
Train Epoch: 2 [44800/60000 (75%)]	 Loss:0.366931
Train Epoch: 2 [51200/60000 (85%)]	 Loss:0.450511
Train Epoch: 2 [57600/60000 (96%)]	 Loss:0.536278
Train Epoch:

/home/hsue/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/home/hsue/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  # This is added back by InteractiveShellApp.init_path()



Test set: Average loss: 0.0017, Accuracy: 9665/10000 (96%)

